In [1]:
import parse

In [2]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=20)

def parses(data):
    return data

data = parses(puzzle.input_data)

In [55]:
from collections import namedtuple
BasePoint = namedtuple("BasePoint", "x, y")

In [56]:
class Point(BasePoint):
    
    def __add__(self, other):
        return Point(self.x+other.x, self.y+other.y)

In [133]:
def build_graph(regex):
    regex = regex.strip('^$')
    graph = defaultdict(set)
    start_stack = []
    end_stack =  []
    # non-deterministic postion list
    current_pos = set([Point(0,0)])
    moves = {'N':Point(-1,0), 
             'S':Point(1,0),
             'E':Point(0,1),
             'W':Point(0,-1)}
    for c in regex:
        if m := moves.get(c, None):
            for p in current_pos:
                graph[p].add(p+m)
                graph[p+m].add(p)
            current_pos = set(p+m for p in current_pos)
        elif c == '(':
            start_stack.append(current_pos.copy())
            end_stack.append(set())
        elif c == '|':
            end_stack[-1] |= current_pos
            current_pos = start_stack[-1].copy()
        elif c == ')':
            start_stack.pop()
            end_stack[-1] |= current_pos
            current_pos = end_stack.pop()
    return graph

In [134]:
build_graph('^WNE$')

defaultdict(set,
            {Point(x=0, y=0): {Point(x=0, y=-1)},
             Point(x=0, y=-1): {Point(x=-1, y=-1), Point(x=0, y=0)},
             Point(x=-1, y=-1): {Point(x=-1, y=0), Point(x=0, y=-1)},
             Point(x=-1, y=0): {Point(x=-1, y=-1)}})

In [135]:
build_graph('^N(E|W)N$')

defaultdict(set,
            {Point(x=0, y=0): {Point(x=-1, y=0)},
             Point(x=-1, y=0): {Point(x=-1, y=-1),
              Point(x=-1, y=1),
              Point(x=0, y=0)},
             Point(x=-1, y=1): {Point(x=-2, y=1), Point(x=-1, y=0)},
             Point(x=-1, y=-1): {Point(x=-2, y=-1), Point(x=-1, y=0)},
             Point(x=-2, y=1): {Point(x=-1, y=1)},
             Point(x=-2, y=-1): {Point(x=-1, y=-1)}})

In [136]:
def render(graph, chars='#|-.'):
    A, B, C, D = chars
    xs = [n.x for n in graph]
    ys = [n.y for n in graph]
    s = A * (2*(max(ys)-min(ys)+1)+1) + '\n'
    for x in range(min(xs), max(xs)+1):
        s += A
        for y in range(min(ys), max(ys)+1):
            z = Point(x,y)
            c = 'X' if z == (0,0) else D
            s += c if z in graph else A
            s += B if z+Point(0,1) in graph[z] else A
        s += '\n' + A
        for y in range(min(ys), max(ys)+1):
            z = Point(x,y)
            s += C if z+Point(1,0) in graph[z] else A
            s += A
        s += '\n'
    print(s)

In [137]:
render(build_graph('WNE'))

#####
#.|.#
#-###
#.|X#
#####



In [138]:
render(build_graph('N(E|W)N'))

#######
#.###.#
#-###-#
#.|.|.#
###-###
###X###
#######



In [139]:
render(build_graph('^ENWWW(NEEE|SSE(EE|N))$'))

#########
#.|.|.|.#
#-#######
#.|.|.|.#
#-#####-#
#.#.#X|.#
#-#-#####
#.|.|.|.#
#########



In [140]:
render(build_graph('^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$'))

###########
#.|.#.|.#.#
#-###-#-#-#
#.|.|.#.#.#
#-#####-#-#
#.#.#X|.#.#
#-#-#####-#
#.#.|.|.|.#
#-###-###-#
#.|.|.#.|.#
###########



In [141]:
print(render(build_graph('WNE')))

#####
#.|.#
#-###
#.|X#
#####

None


In [149]:
from heapq import heappop, heappush
def dijkstra(graph):
    distances = {}
    heap = [(0, Point(0,0))]
    while heap:
        d, pos = heappop(heap)
        if pos in distances:
            continue
        distances[pos] = d
        for neighbor in graph[pos]:
            if neighbor not in distances:
                heappush(heap, (d+1, neighbor))
    return distances

In [ ]:
def solve_a(data):
    distances = dijkstra(build_graph(data))
    return max(distances.values())

def solve_b(data):
    distances = dijkstra(build_graph(data))
    return sum(1 for v in distances.values() if v >= 1000)


In [151]:
solve_b(data)

7956

In [143]:
dijkstra(build_graph('^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$'))

23

In [144]:
dijkstra(build_graph('^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$'))

31

In [148]:
dijkstra(build_graph(data))

3465